In [1]:
%load_ext autoreload
%autoreload 2

In [62]:
import torch
import sophius.utils as utils
import sophius.dataload as dload
from sophius.modelgen import ConvModelGenerator
from sophius.train import train_express_gpu
import torchvision.datasets as dset
import torchvision.transforms as T

In [20]:
VAL_SIZE = 1024

cifar10 = dset.CIFAR10('data//CIFAR10', train=True, download=True,
                           transform=T.ToTensor())
cifar_gpu = dload.cifar_to_gpu(cifar10)

Files already downloaded and verified


In [75]:
model_gen = ConvModelGenerator((3, 32, 32), 10, conv_num=6, lin_num=3)
model_tmpl = model_gen.generate_model_tmpl()
print(model_tmpl)
fixed_model_gpu = model_tmpl.instantiate_model().type(torch.cuda.FloatTensor)

Conv2d       (192, 11, 11)  (4, 4)   (3, 3)  
MaxPool2d    (192, 5, 5)    (3, 3)   (2, 2)  
AvgPool2d    (192, 6, 6)    (2, 2)   (1, 1)  
ReLU         (192, 6, 6)   
Conv2d       (96, 2, 2)     (6, 6)   (4, 4)  
LeakyReLU    (96, 2, 2)     (0.001) 
MaxPool2d    (96, 3, 3)     (4, 4)   (1, 1)  
Conv2d       (256, 1, 1)    (3, 3)   (3, 3)  
PReLU        (256, 1, 1)   
GlobalAvgPool2d (256, 1, 1)   
Flatten      256           
Linear       32            
Dropout2d    32             (0.85)  
LeakyReLU    32             (0.001) 
Linear       10            



In [76]:
fixed_model_gpu

Sequential(
  (0): Conv2d(3, 192, kernel_size=(4, 4), stride=(3, 3), padding=(2, 2))
  (1): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(0, 0), dilation=(1, 1), ceil_mode=False)
  (2): AvgPool2d(kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (3): ReLU()
  (4): Conv2d(192, 96, kernel_size=(6, 6), stride=(4, 4), padding=(3, 3))
  (5): LeakyReLU(negative_slope=0.001)
  (6): MaxPool2d(kernel_size=(4, 4), stride=(1, 1), padding=(2, 2), dilation=(1, 1), ceil_mode=False)
  (7): Conv2d(96, 256, kernel_size=(3, 3), stride=(3, 3), padding=(1, 1))
  (8): PReLU(num_parameters=256)
  (9): AvgPool2d(kernel_size=(1, 1), stride=(1, 1), padding=0)
  (10): Flatten()
  (11): Linear(in_features=256, out_features=32, bias=True)
  (12): Dropout2d(p=0.85, inplace=False)
  (13): LeakyReLU(negative_slope=0.001)
  (14): Linear(in_features=32, out_features=10, bias=True)
)

In [77]:
loader_gpu = dload.get_loader_gpu(cifar_gpu, val_size=VAL_SIZE, batch_size=1024)

# express train
t, val_acc, train_acc = train_express_gpu(model = fixed_model_gpu,
                                          train = True,
                                          loader = loader_gpu,
                                          milestones = [],
                                          num_epoch = 10,
                                          verbose = True)

Finished in 14.7s 
val_acc: 0.457, train_acc: 0.478


In [8]:
print(len(cifar10))

50000


In [9]:
loader = dload.get_loader(cifar10, VAL_SIZE, 1024)
batch = next(iter(loader_gpu['train']))
print(len(batch[0]))
print(type(batch[0]))

1024
<class 'torch.Tensor'>
